In [ ]:
import requests
import os
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime

file_name = 'mobilar.xlsx'

def append_to_csv(fpath, df):
    if (os.path.exists(fpath)):
        x=pd.read_csv(fpath)
    else :
        x=pd.DataFrame()

    dfNew=pd.concat([x,df])
    dfNew['var_perc'] = dfNew.groupby('titulo')['precio_dolar'].pct_change().mul(100).round(2)
    dfNew.to_csv(fpath,index=False)

def append_to_excel(fpath, df):
    if (os.path.exists(fpath)):
        x=pd.read_excel(fpath)
    else :
        x=pd.DataFrame()

    dfNew=pd.concat([x,df])
    dfNew['var_perc'] = dfNew.groupby('titulo')['precio_dolar'].pct_change().mul(100).round(2)
    dfNew.to_excel(fpath,index=False)
            
#limpiador texto andres
def get_clean_text(texto):
    return texto.replace('\n','').replace('.','').replace(',','.').replace('$','')

res2 = requests.get('https://dolarhoy.com')
res2.raise_for_status()
soup2 = bs(res2.text, 'html.parser')
blue = soup2.select("div.venta > div.val")
dolara = blue[0].text.replace('$','')

hoy = datetime.now()
fechab = datetime.strftime(hoy,'%d/%m/%Y')

cate = ['https://www.mobilar.com.ar/t/tv-audio-y-video',
        'https://www.mobilar.com.ar/t/climatizacion',
        'https://www.mobilar.com.ar/t/tecnologia',
        'https://www.mobilar.com.ar/t/hogar',
        'https://www.mobilar.com.ar/t/pequenos-electrodomesticos',
        'https://www.mobilar.com.ar/t/hogar/homedeco']

n = len(cate)
m = 0
url ='https://www.mobilar.com.ar/t/tv-audio-y-video'
while m < n:
    res = requests.get(url)    
    res.raise_for_status()
    if res.status_code ==200:
        soup = bs(res.text, 'html.parser')

        #usando find_all
        headers = soup.find_all(["h3"])
        notebook = [i.text.replace('\t','') for i in headers]
        filtros='Filtrar por'
        if filtros in notebook:
            notebook.remove(filtros)
        notebooks=[b.replace('\n','') for b in notebook]
        
        #precios 
        precio = soup.find_all('span', {'class':"price product-price precio-list"})
        precios_limpio = [i.text for i in precio]
        precios = [get_clean_text(precio) for precio in precios_limpio]

        #usando selectores
        urls = soup.select("h3 > a")
        links = [link['href'] for link in urls]        

        #dataframe y crear o append excel
        df = pd.DataFrame({"fecha":fechab,"titulo":notebooks,"precios":precios,"dolar_hoy":dolara,"url":links})
        df['precios'] = df.precios.astype('float')
        df['dolar_hoy'] = df.dolar_hoy.astype('float')
        df['precio_dolar'] = df['precios']/df['dolar_hoy']           
            
        append_to_excel(file_name,df)
        ja = len(soup.select('li.next_page > a'))
        if ja != 0:
            prevLink = soup.select('li.next_page > a')[0]
            url = 'https://www.mobilar.com.ar' + prevLink.get('href')
            #print(url)
        else:
            m = m +1
            if m != n:
                new_cate = cate[0+m]
                url = new_cate
                print (url)
            else:
                break
        
    
    else:
        break

In [ ]:
import pandas as pd
from datetime import datetime
hoy = datetime.now()
fechab = datetime.strftime(hoy,'%d/%m/%Y')

pd.set_option("display.max_colwidth", None)
df_mobilar = pd.read_excel('mobilar.xlsx')
df_mobilar = df_mobilar[df_mobilar['var_perc'] < -20]
df_mobilar = df_mobilar.loc[df_mobilar.eq(fechab).any(axis=1)]
df_mobilar['var_perc'] = df_mobilar['var_perc'].round(2)

def get_msg(row):
    return f"oferta {row['fecha']} {row['titulo']} esta con {row['var_perc']} % de descuento, en la web : {row['url']} "
msj_mobilar = df_mobilar.apply(lambda row: get_msg(row),axis=1)

for mensaje in msj_mobilar.items():
    mensaje = str(mensaje)
    url = "https://discord.com/api/webhooks/1135358586025955339/yNZpxyaBPx37SUCbd3HaEFKOcKccCIWaudavJtCTNuxbXBNCAveDs3M9Ky-BamPb1drN"
    data = {"username": "Gg","content": mensaje}

    status_discord = requests.post (url, json=data)
    print (status_discord)